# Deep Network for Design Parameters for FWUAV

Importing the Modules

In [1]:
# Modules to import for data preprocessing
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Modules to import for Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check for GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Loading the Data

In [3]:
# Loading the data into the pandas dataframe

# Printing the datafram 
print("\n Data loaded into pandas : \n", data)

# Reading the Data
data = pd.read_csv("Design_Data.csv", header=None)

# Set the first column to object type
data[:0] = data[:0].astype('object')

# Set dtype for the rest of the columns to float32
data.iloc[:, 1:] = data.iloc[:, 1:].astype('float32')

print("\n---------------------------------------------------------------------------\n")

# Inserting the data columns 
data.columns = [
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack', 'Lift', 'Induced Drag'
]

print("\n---------------------------------------------------------------------------\n")

# Split the data into features and targets
X = data[[
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack'
]]

print("Features : \n", X.values)

print("\n---------------------------------------------------------------------------\n")
y = data[['Lift', 'Induced Drag']]

print("Targets : \n",y.values)




 Data loaded into pandas : 
 <module 'torch.utils.data' from '/home/stimp/anaconda3/envs/mlenv/lib/python3.8/site-packages/torch/utils/data/__init__.py'>

---------------------------------------------------------------------------


---------------------------------------------------------------------------

Features : 
 [['naca8304' 0.4000000059604645 1.5 ... 0.4000000059604645 3.0 5.0]
 ['naca8304' 0.4000000059604645 1.5 ... 0.4000000059604645 3.0 15.0]
 ['naca8304' 0.4000000059604645 1.5 ... 0.4000000059604645 3.0 25.0]
 ...
 ['goe225' 0.800000011920929 2.0 ... 1.100000023841858 6.5 35.0]
 ['naca2412' 0.800000011920929 2.0 ... 1.100000023841858 6.5 35.0]
 ['naca0012' 0.800000011920929 2.0 ... 1.100000023841858 6.5 35.0]]

---------------------------------------------------------------------------

Targets : 
 [[-0.18576185405254364 0.010605402290821075]
 [-0.4964708685874939 -0.05340539291501045]
 [-0.8249507546424866 -0.16106195747852325]
 ...
 [-33.38216781616211 -5.5733551979064

One Hot encoder and scaler fit


In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Assuming X and y are already defined in your Jupyter notebook
# X - Features DataFrame
# y - Targets DataFrame

# Drop the Airfoil column from the features
airfoils = X['Airfoil']

# One hot encode the airfoil column
encoder = OneHotEncoder(sparse_output=False)
encoded_airfoil = encoder.fit_transform(airfoils.values.reshape(-1, 1))

print("\nEncoded Airfoils:\n", encoded_airfoil)

X_num = X.drop('Airfoil', axis=1)

Scalerx = MinMaxScaler()
Scalery = MinMaxScaler()

X_scaled_np = Scalerx.fit_transform(X_num)
X_scaled = pd.DataFrame(X_scaled_np, columns=X_num.columns)
print("\nNormalized Features without Airfoil:\n", X_scaled)

y_scaled_np = Scalery.fit_transform(y.values)
y_scaled = pd.DataFrame(y_scaled_np, columns=y.columns)
print("\nNormalized Targets:\n", y_scaled)

X_encoded_pd = pd.DataFrame(encoded_airfoil, columns=encoder.get_feature_names_out(['Airfoil']))
print("\nEncoded Airfoil DataFrame:\n", X_encoded_pd)

X_final = pd.concat([X_encoded_pd, X_scaled], axis=1)
print("\nScaled and Encoded Features:\n", X_final)



Encoded Airfoils:
 [[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

Normalized Features without Airfoil:
        Wing Span  Taper Ratio  Aspect Ratio  Flapping Period  Airspeed  \
0       0.076923     0.076923      0.000000         0.222222  0.222222   
1       0.076923     0.076923      0.000000         0.222222  0.222222   
2       0.076923     0.076923      0.000000         0.222222  0.222222   
3       0.076923     0.076923      0.000000         0.222222  0.222222   
4       0.076923     0.076923      0.000000         0.222222  0.444444   
...          ...          ...           ...              ...       ...   
10682   0.384615     0.230769      0.333333         1.000000  1.000000   
10683   0.384615     0.230769      0.333333         1.000000  1.000000   
10684   0.384615     0.230769      0.333333         1.000000  1.000000   
10685   0.384615     0.230769      0.333333         1.000000  1.000000   
10686   0.384615     0.230769  

# Splitting the data into training and testing 

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y_scaled, test_size=0.2, random_state=42)

# Verify the split
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)


Training features shape: (8549, 10)
Testing features shape: (2138, 10)
Training target shape: (8549, 2)
Testing target shape: (2138, 2)


# Defining the Model

In [6]:
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(10, 128)  # Change input features to 10
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.5)  # Dropout layer
        self.fc4 = nn.Linear(32, 2)    # Change output features to 2
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


In [7]:
# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

In [8]:
print(X_train.shape)
print(y_test)

torch.Size([8549, 10])
tensor([[0.9192, 0.1097],
        [0.9863, 0.0346],
        [0.7682, 0.2483],
        ...,
        [0.9631, 0.0365],
        [0.9796, 0.0491],
        [0.9909, 0.0391]], device='cuda:0')


# Training and eval of the model 

In [13]:
# Move the model to GPU if available
model = DeepNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping
patience = 1000
best_loss = float('inf')
best_model_state = None
no_improvement_epochs = 0

# Training the model
num_epochs = 10000
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if test_loss < best_loss:
        best_loss = test_loss
        best_model_state = model.state_dict()
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}')

# Load the best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)


Epoch [10/10000], Train Loss: 0.21527442336082458, Test Loss: 0.19278553128242493
Epoch [20/10000], Train Loss: 0.09851495921611786, Test Loss: 0.037939514964818954
Epoch [30/10000], Train Loss: 0.09136723726987839, Test Loss: 0.0349259115755558
Epoch [40/10000], Train Loss: 0.0800982415676117, Test Loss: 0.046725936233997345
Epoch [50/10000], Train Loss: 0.07053242623806, Test Loss: 0.025826431810855865
Epoch [60/10000], Train Loss: 0.06253427267074585, Test Loss: 0.029848847538232803
Epoch [70/10000], Train Loss: 0.059218183159828186, Test Loss: 0.023065762594342232
Epoch [80/10000], Train Loss: 0.054506778717041016, Test Loss: 0.023186003789305687
Epoch [90/10000], Train Loss: 0.05192098021507263, Test Loss: 0.02065351977944374
Epoch [100/10000], Train Loss: 0.04812939465045929, Test Loss: 0.018981533125042915
Epoch [110/10000], Train Loss: 0.046312954276800156, Test Loss: 0.01834794320166111
Epoch [120/10000], Train Loss: 0.044734638184309006, Test Loss: 0.017525790259242058
Epoch 

# Saving the Model

In [14]:
torch.save(model.state_dict(), 'avg_flightdata_model.pth')